In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
import shutil
from importlib import reload
import skimage.io
import matplotlib 
import matplotlib.pyplot as plt 
import matplotlib.patches as patches

# local code
import sys
sys.path.append("../source/")

In [2]:
SCALING = 0.7310
MINUTES_PER_FRAME = 15

In [3]:
# DATA PATHS
OUTPUT_DIR = f"/nfs/turbo/umms-indikar/shared/projects/live_cell_imaging/PIP_FUCCI_ANALYSIS/WH18/processed/test_annotations/"
ROOT_DIR = "/nfs/turbo/umms-indikar/shared/projects/live_cell_imaging/PIP_FUCCI_ANALYSIS/WH18/test/"


"""
load dataframes
"""

df_list = []

for f in os.listdir(ROOT_DIR):
    if f.endswith(".csv"):
        f_name = f.split(".")[0]
        f_path = f"{ROOT_DIR}/{f}"
        tmp = pd.read_csv(f_path, skiprows=[1, 2, 3])

        tmp = tmp[tmp['TRACK_ID'].notna()]
        tmp['TRACK_ID'] = f"{f_name}_" + tmp['TRACK_ID'].astype(int).astype(str) 
        tmp['FILE'] = f_name

        df_list.append(tmp)
            
            
            


print(df.shape)
df.head()  

(19746, 47)


,LABEL,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_Z,POSITION_T,FRAME,RADIUS,...,ELLIPSE_Y0,ELLIPSE_MAJOR,ELLIPSE_MINOR,ELLIPSE_THETA,ELLIPSE_ASPECTRATIO,AREA,PERIMETER,CIRCULARITY,SOLIDITY,FILE
0,ID2048,2048,control_1_0,0.870255,350.933240,765.008496,0.0,0.0,0,10.454558,...,0.030824,13.695608,8.077462,1.171379,1.695534,343.369080,73.163177,0.806094,0.960733,control_1
1,ID2049,2049,control_1_1,0.869370,689.637634,1237.841728,0.0,0.0,0,11.421146,...,-0.050914,12.803980,10.286145,0.404286,1.244779,409.797431,76.088193,0.889497,0.975501,control_1
2,ID2050,2050,control_1_2,0.869342,873.457934,1660.396725,0.0,0.0,0,8.850147,...,-0.060996,11.917199,6.698947,-1.201110,1.778966,246.065580,63.521177,0.766344,0.949458,control_1
3,ID2051,2051,control_1_3,0.869014,495.260834,222.409189,0.0,0.0,0,9.436441,...,-0.010069,11.527043,7.822970,0.827377,1.473487,279.747561,64.185006,0.853314,0.961415,control_1
4,ID2052,2052,control_1_4,0.868918,1597.010688,1188.943342,0.0,0.0,0,8.167641,...,0.012290,9.127027,7.418472,-0.785027,1.230311,209.576768,55.557312,0.853239,0.965517,control_1


In [4]:
"""
load images
"""

files = {}


for f in os.listdir(ROOT_DIR):
    if f.endswith(".tif"):
            f_name = f.split(".")[0]
            f_path = f"{ROOT_DIR}/{f}"
            tiff = skimage.io.imread(f_path)
            files[f_name] = tiff
                
                
files.keys()


dict_keys(['control_1'])

In [5]:
# a few quick exclusions based on size and track length

min_track_length = 10
min_mean_size = 2

print(f"{df.shape=}")
print(f"{df['TRACK_ID'].nunique()=}")

df = df.sort_values(by=['TRACK_ID', 'FRAME'])

df['STEP'] =  df.groupby('TRACK_ID').transform('cumcount')
df['MAX_STEP'] =  df.groupby('TRACK_ID')['STEP'].transform('max')


####### perform all filtering on a temporary dataframe
tmp = df.copy()

filtered = df.groupby('TRACK_ID')['STEP'].max().reset_index()
filtered = filtered[filtered['STEP'] > min_track_length]
tmp = tmp[tmp['TRACK_ID'].isin(filtered['TRACK_ID'])]

print(f"\nfilter number of frames:")
print(f"\t{tmp.shape=}")
print(f"\t{tmp['TRACK_ID'].nunique()=}")

filtered = tmp.groupby('TRACK_ID')['RADIUS'].mean().reset_index()
filtered = filtered[filtered['RADIUS'] > min_mean_size]
tmp = tmp[tmp['TRACK_ID'].isin(filtered['TRACK_ID'])]

print(f"\nfilter nucleus size:")
print(f"\t{tmp.shape=}")
print(f"\t{tmp['TRACK_ID'].nunique()=}")


"""
Filter values by their maximum min-green intensity
If the cell never expressed green, it is not considered
"""


df = tmp.copy()
df = df.sort_values(by=['TRACK_ID', 'FRAME', "FILE"])

filename = f"{ROOT_DIR}filtered_spots.csv"

df.to_csv(filename, index=False)
print(f"{filename=}")

df.shape=(19746, 47)
df['TRACK_ID'].nunique()=580

filter number of frames:
	tmp.shape=(19113, 49)
	tmp['TRACK_ID'].nunique()=470

filter nucleus size:
	tmp.shape=(19113, 49)
	tmp['TRACK_ID'].nunique()=470
filename='/nfs/turbo/umms-indikar/shared/projects/live_cell_imaging/PIP_FUCCI_ANALYSIS/WH18/test/filtered_spots.csv'


In [6]:
def get_images(tiff, frame, xpos, ypos, xwin=50, ywin=50):
    """A function to return a small frame of from a position
    at a specific time from a movie
    
    expects input shape (time, y, x, 3) for RGB image
    """
    f, m, n, c = tiff.shape
    
    xmin = int(xpos - xwin)
    xmax = int(xpos + xwin)
    ymin = int(ypos - ywin)
    ymax = int(ypos + ywin)
    
    spot_x, spot_y = xwin, ywin
    
    # handle boundaries
    if xmin < 0:
        spot_x = spot_x + xmin
        xmin = 0 
    
    if xmax > n:
        xmax = n
        
    if ymin < 0:
        spot_y = spot_y + ymin
        ymin = 0

    if ymax > m:
        ymax = m

    img = tiff[frame, ymin:ymax, xmin:xmax, :]
    return img, (spot_x, spot_y)


In [7]:
N_TRACKS = df['TRACK_ID'].nunique()
print(f"{N_TRACKS=}")


PROPORTION_OF_TRACKS = 1
SAMPLE_SIZE = int(N_TRACKS * PROPORTION_OF_TRACKS) # this is the number of TRACKS to be sampled
print(f"{SAMPLE_SIZE=}\n")

N_TRACKS=470
SAMPLE_SIZE=470



In [ ]:
break

In [8]:
# store images for annotation

xwin = 100
ywin = 100
nucleus_size = 30
plt.rcParams['figure.dpi'] = 150
plt.rcParams['font.size'] = 6
plt.rcParams['figure.figsize'] = 4,4

# will overwrite the dir each time
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR)

# np.random.seed(0)
tracks = np.random.choice(df['TRACK_ID'].unique(), SAMPLE_SIZE)
tmp = df[df['TRACK_ID'].isin(tracks)]

print(f"{tmp.shape=}")

last_track = None
for idx, row in tmp.iterrows():
    xpos = row['POSITION_X'] * SCALING
    ypos = row['POSITION_Y'] * SCALING
    
    frame = int(row['FRAME'])
    track = row['TRACK_ID']
    
    if not track == last_track:
        print(f"{track=}")
    last_track = track
    
    # make sure we always have a left and right image
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
    
    tiff = files[row["FILE"]]

    # image under question 
    img, spot = get_images(tiff, frame, xpos, ypos, xwin, ywin)
    
    if len(img) == 0:
        continue
    
    ax1.imshow(img)
    ax1.axis('off')
    spot_blob = plt.Circle(spot, nucleus_size, color='r', fill=False)
    ax1.add_patch(spot_blob)
    
    ax2.imshow(img[:, :, 1], cmap='Greens')
    ax2.axis('off')
    spot_blob = plt.Circle(spot, nucleus_size, color='r', fill=False)
    ax2.add_patch(spot_blob)
    
    ax3.imshow(img[:, :, 0], cmap='Reds')
    ax3.axis('off')
    spot_blob = plt.Circle(spot, nucleus_size, color='r', fill=False)
    ax3.add_patch(spot_blob)
    
    plt.title(f"{track=} {frame=}")
    
    frame_fname = str(frame).zfill(4)
           
    filename = f"{track}_frame_{frame_fname}.png"
    save_path = f"{OUTPUT_DIR}{filename}"
    
    plt.savefig(save_path,  bbox_inches='tight')
    plt.close(fig)
    
print("\nDone.")

tmp.shape=(11214, 49)
track='control_1_1'
track='control_1_10'
track='control_1_100'
track='control_1_101'
track='control_1_103'
track='control_1_104'
track='control_1_105'
track='control_1_106'
track='control_1_107'
track='control_1_108'
track='control_1_109'
track='control_1_11'
track='control_1_110'
track='control_1_112'
track='control_1_116'
track='control_1_117'
track='control_1_119'
track='control_1_12'
track='control_1_121'
track='control_1_126'
track='control_1_128'
track='control_1_13'
track='control_1_130'
track='control_1_131'
track='control_1_136'
track='control_1_137'
track='control_1_138'
track='control_1_14'
track='control_1_140'
track='control_1_141'
track='control_1_143'
track='control_1_146'
track='control_1_148'
track='control_1_15'
track='control_1_150'
track='control_1_153'
track='control_1_154'
track='control_1_159'
track='control_1_160'
track='control_1_163'
track='control_1_165'
track='control_1_170'
track='control_1_172'
track='control_1_173'
track='control_1_1